In [8]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [12]:
%load_ext rpy2.ipython
%load_ext memory_profiler

### Download and unzip the data

In [9]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "daily_rain/"

In [13]:
%%time
%memit
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

peak memory: 1976.04 MiB, increment: 0.26 MiB
CPU times: user 98.9 ms, sys: 132 ms, total: 231 ms
Wall time: 4.52 s


[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [14]:
%%time
%memit
files_to_dl = ["data.zip"] 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

peak memory: 1979.29 MiB, increment: 0.03 MiB
CPU times: user 4.41 s, sys: 3.88 s, total: 8.29 s
Wall time: 1min 9s


In [16]:
%%time
%memit
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

peak memory: 859.73 MiB, increment: 0.01 MiB
CPU times: user 15.7 s, sys: 2.29 s, total: 18 s
Wall time: 21.7 s


### Combine csv files

In [4]:
df = pd.read_csv("daily_rain/ACCESS-CM2_daily_rainfall_NSW.csv", dtype={'TailNum': 'str'})

In [8]:
df.columns

Index(['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)'], dtype='object')

In [ ]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 
import pandas as pd
use_cols = ['time', 'rain (mm/day)']
files = glob.glob('daily_rain/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(year=re.findall(r'[^\/]+(?=\.)', file)[0])
                for file in files)
              )
df.to_csv("daily_rain/combined_data.csv")

In [6]:
df = pd.read_csv("daily_rain/combined_data.csv", dtype={'TailNum': 'str'})

In [7]:
df

,time,rain (mm/day),year
0,1889-01-01 12:00:00,4.244226e-13,MPI-ESM-1-2-HAM_daily_rainfall_NSW
1,1889-01-02 12:00:00,4.217326e-13,MPI-ESM-1-2-HAM_daily_rainfall_NSW
2,1889-01-03 12:00:00,4.498125e-13,MPI-ESM-1-2-HAM_daily_rainfall_NSW
3,1889-01-04 12:00:00,4.251282e-13,MPI-ESM-1-2-HAM_daily_rainfall_NSW
4,1889-01-05 12:00:00,4.270161e-13,MPI-ESM-1-2-HAM_daily_rainfall_NSW
...,...,...,...
62513858,2014-12-27 12:00:00,6.689683e+00,SAM0-UNICON_daily_rainfall_NSW
62513859,2014-12-28 12:00:00,7.862555e+00,SAM0-UNICON_daily_rainfall_NSW
62513860,2014-12-29 12:00:00,1.000503e+01,SAM0-UNICON_daily_rainfall_NSW
62513861,2014-12-30 12:00:00,8.541592e+00,SAM0-UNICON_daily_rainfall_NSW
